# 🦜🔗 LangChain核心源代码解读

# ❤️ 课程开始

## 这节课会带给你

1. 🌹阅读 Langchain 的大模型实现源码
2. 🌹阅读 Langchain 的核心组件源码：Runnable及其子类
3. 🌹阅读 LCEL 的智能体实现源码：create_react_agent
4. 🌹阅读 Langgraph 的智能体实现源码：prebuild
5. ✍️ 动手集成自己的大模型到 LangChain：智谱AI
6. ✍️ 动手实现基于 LCEL 的智能体：再现手撕AutoGPT
7. ✍️ 动手实现基于 Langgraph 的智能体：再现手撕AutoGPT

<div class="alert alert-warning">
<b>注意：</b><br>
<ul>
    <li>大模型认知：请参考AGI课堂正课相关章节</li>
    <li>Langchain 基础：请参考AGI课堂正课相关章节</li>
    <li>看懂源码是为了写好代码</li>
</ul> 
</div>

## 如何解读 Langchain 的源代码结构

### 1、你有哪些资源可以利用？

![](./langchain_ai.png)

（1）学习资源：
- [langchain源代码](https://github.com/langchain-ai/)：All you need r here !!!
- [langchain官方文档](https://python.langchain.com/docs)：与源代码相互印证
- [langgraph example](https://github.com/langchain-ai/langgraph/tree/main/examples)：Jupyter Notes

（2）良师益友：
- langchain [聊天](https://chat.langchain.com/?llm=anthropic_claude_2_1)：免费的大模型+RAG（也可以学习其源码）
- Github Copilot：程序员无法离开的工具，就像现在的人开车无法离开地图导航

### 2、🌹 阅读源码：LangChain 源代码概览

[langchain源代码结构](https://github.com/langchain-ai/langchain/tree/master/libs)

| 源码位置 | 功能描述 |
| :--- | :--- |
| langchain/libs/langchain | 模块入口，会导入core、community等其他模块 |
| langchain/libs/core | 核心组件和关键的基类实现 |
| langchain/libs/partners | 合作伙伴（官方合作）组件 |
| langchain/libs/community | 社区（非官方）组件 |
| langchain/libs/experimental | 试验性功能（前沿探索组件，不对版本稳定做承诺） |


### 3、LangChain核心框架的三轮迭代

- Runnable + Chain 时代
- Runnable + LCEL 时代
- Runnable + LCEL + Langgraph 时代

# ❤️ （一）从零开始集成大模型到 Langchain 实例

In [2]:
# 加载 .env 到环境变量
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

## 一般用法

In [17]:
# LLM
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [9]:
# invoke
text = "请帮我想一想，生产彩色铅笔的公司有什么好名字?"
response = llm.invoke(text)
print(response.content)

1. 彩虹铅笔有限公司
2. 色彩创意铅笔厂
3. 彩绘铅笔制造厂
4. 彩色梦想铅笔公司
5. 绚丽色彩铅笔厂家
6. 彩色涂鸦铅笔工作室
7. 花样彩色铅笔制造有限公司
8. 彩绘天地铅笔厂
9. 艳丽色彩铅笔生产厂家
10. 彩虹艺术铅笔公司


In [52]:
# stream
for chunk in llm.stream(text):
    print(chunk.content, end="|", flush=True)

|1|.| 色|彩|乐|园|
|2|.| 彩|虹|铅|笔|厂|
|3|.| 彩|色|创|意|
|4|.| |五|彩|铅|笔|坊|
|5|.| 彩|色|笔|画|
|6|.| 色|彩|世|界|
|7|.| 魔|法|彩|铅|
|8|.| 彩|色|笔|墨|
|9|.| 绚|丽|铅|笔|厂|
|10|.| 彩|虹|色|笔||

In [53]:
# Prompt
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("请帮我想一想，生产{product}的公司有什么好名字?")
prompt.format(product="彩色铅笔")

'请帮我想一想，生产彩色铅笔的公司有什么好名字?'

In [54]:
# LCEL LLM+Prompt
chain = prompt | llm
for chunk in chain.stream({"product": "彩色铅笔"}):
    print(chunk.content, end="|", flush=True)

|1|.| 彩|虹|铅|笔|有|限|公司|
|2|.| 彩|色|创|意|铅|笔|制|造|厂|
|3|.| 缤|纷|彩|铅|笔|有|限|责|任|公司|
|4|.| 色|彩|世|界|铅|笔|制|造|有|限|公司|
|5|.| 梦|幻|色|彩|铅|笔|有|限|公司|
|6|.| 彩|色|创|意|铅|笔|工|作|室|
|7|.| 彩|绘|铅|笔|有|限|公司|
|8|.| 彩|色|艺|术|铅|笔|制|造|厂|
|9|.| 彩|色|创|意|铅|笔|有|限|责|任|公司|
|10|.| 彩|色|笔|墨|有|限|公司||

In [55]:
# LCEL LLM+Prompt+Outputparser
from langchain.schema.output_parser import StrOutputParser

chain = prompt | llm | StrOutputParser()
for chunk in chain.stream({"product": "彩色铅笔"}):
    print(chunk, end="|", flush=True)

|1|.| 彩|虹|铅|笔|工|坊|
|2|.| 色|彩|世|界|铅|笔|公司|
|3|.| 彩|色|创|意|铅|笔|厂|
|4|.| 彩|绘|铅|笔|制|造|厂|
|5|.| 彩|色|笔|芯|工|艺|厂|
|6|.| 彩|铅|笔|创|意|工|坊|
|7|.| |五|彩|铅|笔|制|造|公司|
|8|.| 彩|色|笔|芯|创|意|工|厂|
|9|.| 绚|丽|铅|笔|工|艺|厂|
|10|.| 彩|虹|笔|芯|制|造|厂||

## 实现一个简单的模拟大模型

### 实现一个Fake大模型

In [74]:
import time
from typing import Any, Dict, Iterator, List, Optional, Union
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import HumanMessage, HumanMessageChunk, AIMessage, AIMessageChunk, BaseMessage
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult

In [165]:
class FakeChatWithOlder(BaseChatModel):
    """模拟跟大爷的对话"""

    responses: List[BaseMessage]
    sleep: Optional[float] = 0.1
    i: int = 0

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:
        response = self.responses[self.i]
        if self.i < len(self.responses) - 1:
            self.i += 1
        else:
            self.i = 0
        generation = ChatGeneration(message=response)
        return ChatResult(generations=[generation])

    @property
    def _llm_type(self) -> str:
        return "fake-chat-with-older"

In [166]:
questions = [
    "大爷，楼上322马冬梅在家吗？",
    "马冬梅啊",
    "马冬梅！",
    "大爷您歇着吧..."
]

reply = [AIMessage(m) for m in [
    "马什么梅？",
    "什么冬梅？",
    "马东什么？",
    "好咧"
]]

llm_fake = FakeChatWithOlder(responses=reply)

for question in questions:
    print(f"\n\n夏洛：{question}")
    print("大爷：", end="")
    # print(llm_fake.invoke(question).content, end="")
    for chunk in llm_fake.stream(question):
        print(chunk.content, end="|")



夏洛：大爷，楼上322马冬梅在家吗？
大爷：马什么梅？|

夏洛：马冬梅啊
大爷：什么冬梅？|

夏洛：马冬梅！
大爷：马东什么？|

夏洛：大爷您歇着吧...
大爷：好咧|

### 支持流

In [167]:
class FakeStreamChatWithOlder(FakeChatWithOlder):
    """模拟跟大爷的对话，支持流"""

    def _stream(
        self,
        messages: List[BaseMessage],
        stop: Union[List[str], None] = None,
        run_manager: Union[CallbackManagerForLLMRun, None] = None,
        **kwargs: Any,
    ) -> Iterator[ChatGenerationChunk]:
        response = self.responses[self.i]
        if self.i < len(self.responses) - 1:
            self.i += 1
        else:
            self.i = 0
        for chunk in response.content:
            if self.sleep is not None:
                time.sleep(self.sleep)
            yield ChatGenerationChunk(message=AIMessageChunk(content=chunk))

In [168]:
llm_fake = FakeStreamChatWithOlder(responses=reply)

for question in questions:
    print(f"\n\n夏洛：{question}")
    print("大爷：", end="")
    for chunk in llm_fake.stream(question):
        print(chunk.content, end="|")



夏洛：大爷，楼上322马冬梅在家吗？
大爷：马|什|么|梅|？|

夏洛：马冬梅啊
大爷：什|么|冬|梅|？|

夏洛：马冬梅！
大爷：马|东|什|么|？|

夏洛：大爷您歇着吧...
大爷：好|咧|

### 🌹 看源码

### Langchain支持的大模型
- ChatOpenAI
- FakeLLM

### 一般用法

- invoke
- stream
- create_qa_chain
- create_openai_agent

## 版本1

### 看看智谱官方的简单例子
### Langchain 相关库源码
### 写一个简单的 Langchain 版本
### 在LCEL中使用

## 版本2

### 试试流：暂时不支持
### 看看智谱官方对流的支持
### 看看流的默认实现源码
### 让我们也支持流

## 版本3

### 试试工具：暂时不支持
### 看看智谱官方对工具回调的支持
### 回顾工具回调：只是一种消息格式
### 看看 openai 实现源码
### 让我们也支持工具回调
### 在智能体中使用
### 完整代码请参考 langchain_zhipu

## 1、✍️ 代码准备：langchain中的LLM

In [ ]:
# LLM invoke
# OpenAI

# LLM stream
# OpenAI


### 2、✍️ 代码实践：如何实现一个FakeLLM

In [2]:
# 马冬梅楼下老大爷

[已经实现的FakeLLM](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/language_models/fake.py)

### 3、🌹 阅读源码：集成自己的大模型需要如何下手？

#### （1）BaseLanguageModel

来自：[https://github.com/langchain-ai/langchain](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/language_models/base.py#L74-L81)

```python
class BaseLanguageModel(
    RunnableSerializable[LanguageModelInput, LanguageModelOutputVar], ABC
):
    """Abstract base class for interfacing with language models.


    All language model wrappers inherit from BaseLanguageModel.
    """

    
    ...
```

#### （2）BaseChatModel

来自：[https://github.com/langchain-ai/langchain](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/language_models/chat_models.py#L100)

```python
class BaseChatModel(BaseLanguageModel[BaseMessage], ABC):
    """Base class for Chat models."""

    ...

    def invoke(...) -> BaseMessage:
        # ...
        self.generate_prompt(...)
        # generate_prompt >> generate >> _generate

    def ainvoke(...) -> BaseMessage:
        # ...
        self.agenerate_prompt(...)
        # agenerate_prompt >> agenerate >> _agenerate >> _generate
    
    def stream(...) -> Iterator[BaseMessageChunk]:
        # ...
        if type(self)._stream == BaseChatModel._stream:
            # model doesn't implement streaming, so use default implementation
            yield cast(
                BaseMessageChunk, self.invoke(input, config=config, stop=stop, **kwargs)
            )
        # ...

    async def astream(...) -> AsyncIterator[BaseMessageChunk]:
        # ...
        if (
            type(self)._astream is BaseChatModel._astream
            and type(self)._stream is BaseChatModel._stream
        ):
            # No async or sync stream is implemented, so fall back to ainvoke
            yield cast(
                BaseMessageChunk,
                await self.ainvoke(input, config=config, stop=stop, **kwargs),
            )
        # ...

    ## ！！必须实现 
    @abstractmethod
    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:
        """Top Level call"""

    ## ！！建议实现 
    def _stream(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> Iterator[ChatGenerationChunk]:
        raise NotImplementedError()

    async def _astream(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[AsyncCallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> AsyncIterator[ChatGenerationChunk]:
    # ...
    self._stream(...)
        
```

### 4、✍️ 代码实践：一步步集成智谱AI大模型

- 支持 invoke
- 支持 stream
- 支持 tools-calling

- 完整实现：[langchain_zhipu](https://github.com/arcstep/langchain_zhipu)

# ❤️（二）LLM输入输出：全面拆解 Runnable 结构

### 1、大模型调用过程中发生的数据流转换

- 从训过的模型说起：训练预料和预测文本风格（|<assistent>| xxxxxx |</assistent>|)
- 作为API提供：openAI风格
- 从提示语到langchain消息：BaseMessages
- 从langchain消息到大模型客户端：BaseChatModel
- 从大模型结果到langchain消息：BaseMessages
- 从langchain消息到输出解析器：OutpurParser

### 2、如果在这个过程中统一各模块标准，随时更换？

### 3、如果在这个过程中使用流、异步、批量、事件流 ... ？

### 4、这个过程中的缓存、重试、配置 ... ？

### 1、为什么说可以用 Runnable 组件在生产系统中搭积木？

- Runnable接口标准化的价值
    - 替换大模型：从GPT到国产云模型、到开源自训模型的多层次尝试和落地是必要的
    - 替换调用方式：满足业务验证、生产上线、业务扩容等多场景
    - 替换提示语模板：适应技术研究、业务定制、运营优化等多阶段
    - 替换解析器：适应模块调用、API调用等多协议对接
    - 替换回调集成：langsmith、langfuse云、langfuse本地化、自建运维系统
    - 替换向量数据库：适应不同场景、阶段的技术选型调整
    - 替换持久化：...
    - 替换智能体：...

### 2、🌹 阅读源码：Runnable 组件8个方法的实现逻辑

### 3、🌹 阅读源码：Runnable 组件配置自举能力

- 了解Runnable 组件的 8 个方法的默认实现
- 自定义Runnable（必要实现一般是invoke / stream / astream，或对应的内部函数）
- invoke：标准化调度
- batch: 标准化批量调度
- stream：标准化流式输出
- ainvoke / astream / abatch: 标准化异步调度
- astream_log / astream_events: 在链、智能体、langgraph等输出中按照names、tags、events提取流式日志
- config：统一管理配置
- schema：统一探查参数和配置

- 序列化

- 配置自举：在开放式应用中支持客户端自动识别自定义服务
- 容错：标准化重试策略
- 与langserve等api框架标准化对接
- 与langfuse等callbck框架标准化对接
- 与langchainjs等异构实现标准化对接


### 4、🌹 阅读源码：遗留的 Chain 是什么？

- langchain中提前写好的Chain资源

- 这些 Chain 局限性在究竟哪里？
    - 流程不灵活
    - 支持流式输出不彻底

### 5、✍️ 代码实践：若干种情况下的流输出

# ❤️ 第 2 部份：Agent

# ❤️（三）由LCEL实现智能体：全面拆解 LCEL 能力

### 1、LCEL 比 遗留 Chain 多哪些优势？

- LCEL 构建的替代 Chain
- 了解支撑LCEL的Runnable组件
    - Lambda
    - 迭代器
    - 字典和并行
    - 路由
    - 条件
    - 迭代执行
    - 绑定
    - 绘图

### 2、如何用 LCEL 定义智能体？

- 工具：定义一个简单工具
- 智能体：
    - Tools-Calling 智能体
    - ReAct 智能体
- 执行器：AgentExecutor

### 3、🌹 阅读源码：了解 create_react_agent 的设计

- 一个简单的智能体需求：与AI玩一个捉迷藏游戏
- react智能体的prompt如何工作
- 中间步骤的一步步产生过程
- action如何解析

### 4、🌹 阅读源码：了解 AgentExecutor 的执行逻辑

### 4、✍️ 代码实践：如何用 AgentExcutor 再现《手撕AutoGPT》？

难点：
- 官方例子和内置智能体无法支持pydantic参数解析（智谱AI等推理能力较弱的模型可以使用）

# ❤️（四）由LangGraph实现智能体：全面拆解 LangGraph 能力

### 1、LangGraph 比 LCEL 多了什么？

### 2、如何使用 LangGraph 定义智能体？

### 3、🌹 阅读源码：了解 LangGraph 的执行逻辑

### 4、✍️ 代码实践：如何用 LangGraph 再现《手撕AutoGPT》？

难点：
- 官方例子和内置智能体无法支持流

# ❤️ 课程结束

## 1、课程总结

- 我们一起阅读了langchain的源代码结构和部份细节
    - BaseLanguageModel / BaseChatModel / 
    - Runnable
    - LambdaRunnable
    - Chain
    - AgentExcutor
    - langgraph.prebuild
- 我们学习了如何自己动手集成大模型到 langchain 中
- 我们拆解了langchain的基石组件：Runnable
- 我们拆解了langchain的核心逻辑能力：LCEL
- 我们拆解了langchain的最新逻辑能力：langgraph
- 我们动手做了一些代码实践

## 2、最后建议

- 技术选型时要对 Langchain 有绝对信心（几乎都不会是langchain的错）
- 内置链尽量使用LCEL链
- 内置智能体尽量使用 Langgraph
- 自定义智能体时使用 Langgraph
- 模块优先做成Runnable或LCEL链，其次再考虑Lambda
- 工具中包含大模型调用时优先做成Runnable或LCEL链，其次再考虑invoke


## 3、彩蛋

### 1、✍️ 代码实践：如何同时使用langchain的记忆和持久化

这是 langchain 文档中一个自相矛盾的地方，留给大家课后讨论。

### 2、✍️ 代码实践：如何将自己训的大模型集成到 langchain 中